# Build custom container for Vertex training

In [1]:
!pwd

/home/jupyter/tf_vertex_agents/04-perarm-features-bandit


## Load env config

* use the prefix from `00-env-setup`

In [2]:
PREFIX = 'mabv1'

In [3]:
# staging GCS
GCP_PROJECTS             = !gcloud config get-value project
PROJECT_ID               = GCP_PROJECTS[0]

# GCS bucket and paths
BUCKET_NAME              = f'{PREFIX}-{PROJECT_ID}-bucket'
BUCKET_URI               = f'gs://{BUCKET_NAME}'

config = !gsutil cat {BUCKET_URI}/config/notebook_env.py
print(config.n)
exec(config.n)


PROJECT_ID               = "wortz-project-352116"
PROJECT_NUM              = "679926387543"
LOCATION                 = "us-central1"

REGION                   = "us-central1"
BQ_LOCATION              = "US"
VPC_NETWORK_NAME         = "ucaip-haystack-vpc-network"

VERTEX_SA                = "679926387543-compute@developer.gserviceaccount.com"

PREFIX                   = "mabv1"
VERSION                  = "v1"

BUCKET_NAME              = "mabv1-wortz-project-352116-bucket"
BUCKET_URI               = "gs://mabv1-wortz-project-352116-bucket"
DATA_GCS_PREFIX          = "data"
DATA_PATH                = "gs://mabv1-wortz-project-352116-bucket/data"
VOCAB_SUBDIR             = "vocabs"
VOCAB_FILENAME           = "vocab_dict.pkl"

VPC_NETWORK_FULL         = "projects/679926387543/global/networks/ucaip-haystack-vpc-network"

BIGQUERY_DATASET_ID      = "wortz-project-352116.movielens_dataset_mabv1"
BIGQUERY_TABLE_ID        = "wortz-project-352116.movielens_dataset_mabv1.training_dataset"

REPO_D

In [4]:
! gsutil ls $BUCKET_URI

gs://mabv1-wortz-project-352116-bucket/config/
gs://mabv1-wortz-project-352116-bucket/data/
gs://mabv1-wortz-project-352116-bucket/generative-trajectories/
gs://mabv1-wortz-project-352116-bucket/neural-linear-bandits-v1/
gs://mabv1-wortz-project-352116-bucket/vocabs/


## imports

In [5]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Build Image

## Container Image Variables

In [6]:
DOCKERNAME                = f'Dockerfile_perarm_feats'

REPOSITORY                = f'rl-movielens-{PREFIX}'
IMAGE_NAME                = f'train-perarm-feats-{VERSION}'

REMOTE_IMAGE_NAME         = f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE_NAME}"

print(f"DOCKERNAME        = {DOCKERNAME}")
print(f"REPOSITORY        = {REPOSITORY}")
print(f"IMAGE_NAME        = {IMAGE_NAME}")
print(f"REMOTE_IMAGE_NAME = {REMOTE_IMAGE_NAME}")

DOCKERNAME        = Dockerfile_perarm_feats
REPOSITORY        = rl-movielens-mabv1
IMAGE_NAME        = train-perarm-feats-v1
REMOTE_IMAGE_NAME = us-central1-docker.pkg.dev/wortz-project-352116/rl-movielens-mabv1/train-perarm-feats-v1


## Create Artifact Repository

If you don't have an existing artifact repository, create one using the gcloud command below

In [7]:
# ! gcloud artifacts repositories create $REPOSITORY --repository-format=docker --location=$LOCATION

## Local Docker build

Provide a name for your dockerfile and make sure you are authenticated

In [8]:
! gcloud auth configure-docker $REGION-docker.pkg.dev --quiet


{
  "credHelpers": {
    "gcr.io": "gcloud",
    "us.gcr.io": "gcloud",
    "eu.gcr.io": "gcloud",
    "asia.gcr.io": "gcloud",
    "staging-k8s.gcr.io": "gcloud",
    "marketplace.gcr.io": "gcloud",
    "us-central1-docker.pkg.dev": "gcloud"
  }
}
Adding credentials for: us-central1-docker.pkg.dev
gcloud credential helpers already registered correctly.


### Create Dockerfile

In [9]:
import os

root_path = '..'
os.chdir(root_path)
os.getcwd()

'/home/jupyter/tf_vertex_agents'

In [46]:
%%writefile Dockerfile

FROM gcr.io/deeplearning-platform-release/base-cu113.py310

#updated to be a derivative image - good for general GCP usage

ENV PYTHONUNBUFFERED True

ENV APP_HOME /workspace

WORKDIR $APP_HOME

# COPY requirements.txt $APP_HOME/requirements.txt
COPY /requirements.txt $APP_HOME/requirements.txt

RUN pip install --upgrade pip
RUN pip install --no-cache-dir -r $APP_HOME/requirements.txt

RUN ls $APP_HOME

# ADD src $APP_HOME/src
# RUN ls src
COPY src/perarm_features $APP_HOME/src/perarm_features
COPY src/per_arm_rl $APP_HOME/src/per_arm_rl

RUN ls $APP_HOME

RUN export PYTHONPATH=${PYTHONPATH}:${APP_HOME}/

# Sets up the entry point to invoke the task.
ENTRYPOINT ["python3", "-m", "src.perarm_features.task"]

Overwriting Dockerfile


### Build Image Locally

In [ ]:
!docker build -t $REMOTE_IMAGE_NAME -f Dockerfile .

Sending build context to Docker daemon  117.7MB
Step 1/13 : FROM gcr.io/deeplearning-platform-release/base-cu113.py310
latest: Pulling from deeplearning-platform-release/base-cu113.py310

351b9876: Pulling fs layer 
fdd4cb87: Pulling fs layer 
33b03941: Pulling fs layer 
420ffd93: Pulling fs layer 
4dad10e9: Pulling fs layer 
78304786: Pulling fs layer 
f3f1d840: Pulling fs layer 
9842b228: Pulling fs layer 
cdc05756: Pulling fs layer 
b08ca694: Pulling fs layer 
2cb29222: Pulling fs layer 
83c5e84f: Pulling fs layer 
a3a5047e: Pulling fs layer 
0a22815b: Pulling fs layer 
b700ef54: Pulling fs layer 
c8d1312d: Pulling fs layer 
df4e233c: Pulling fs layer 
2cdf5f7e: Pulling fs layer 
ecb0700e: Pulling fs layer 
bdf3320f: Pulling fs layer 
22039c10: Pulling fs layer 
b71c7b87: Pulling fs layer 
213d239d: Pulling fs layer 
cb2fbb76: Pulling fs layer 
32f9e341: Pulling fs layer 
dd23c9e6: Pulling fs layer 
3243375c: Pulling fs layer 
53050b2e: Pulling fs layer 
e36971b8: Pulling fs layer 


In [30]:
# !gcloud builds submit ./ --region $REGION --tag $REMOTE_IMAGE_NAME 

Creating temporary tarball archive of 98 file(s) totalling 86.0 MiB before compression.
Uploading tarball of [./] to [gs://wortz-project-352116_cloudbuild/source/1692588979.095783-ca5671936edb4440bf3a1ff97920132d.tgz]
ERROR: (gcloud.builds.submit) PERMISSION_DENIED: The caller does not have permission


### Push container to Registry

In [31]:
# ### push the container to registry
!docker push $REMOTE_IMAGE_NAME

Using default tag: latest
The push refers to repository [us-central1-docker.pkg.dev/wortz-project-352116/rl-movielens-mabv1/train-perarm-feats-v1]

e4407469: Preparing 
13137420: Preparing 
f052da70: Preparing 
29ac24a3: Preparing 
c78b3f70: Preparing 
9ffc4ad6: Preparing 
1a3b336f: Preparing 
4990d0ba: Preparing 
db13266b: Preparing 
b1eee192: Preparing 
c6cd33a6: Preparing 
72026017: Preparing 
18a5eaf8: Preparing 
a9b258e5: Layer already exists latest: digest: sha256:3b4428cbd993a6fcc704a7c3d6504f3d0c621045e849c4533342ff63cbb83f38 size: 3264
